# Setup

In [ ]:
# Enable automatic reloading of modules
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os

# Ensure the project level directory is the working directory
if "DIR_CHANGED" in locals():
    print("Directory already changed")
else:
    project_dir = os.path.join(os.path.abspath("") + "/..")
    os.chdir(project_dir)
    DIR_CHANGED = True

In [ ]:
from requests import post
import pandas as pd
import plotly.express as px
from statsmodels.tsa.ar_model import AutoReg
from matplotlib import pyplot as plt

from src.utils import load_config
from src.data_loading import load_house_prices

config = load_config()

In [ ]:
df = load_house_prices(config)

# Analysis

In [ ]:
df["quarter"] = df["kvartal"].replace("K", "Q", regex=True)
df["quarter"] = pd.PeriodIndex(df["quarter"], freq="Q-DEC").to_timestamp(freq="Q-DEC")
df = df.set_index("quarter")
df.index.freq = "Q-DEC"

## Quarterly house prices

In [ ]:
fig = px.line(
    df["value"],
    labels={"quarter": "Date", "_value": "Price per m<sup>2</sup>"},
    title="Price index for existing dwellings",
)
fig.show(renderer="notebook")

In [ ]:
def evaluate_model(model, df):
    """Plot standard diagnostics + model predictions"""
    res = model.fit()
    fig = plt.figure(figsize=(16, 9))
    fig = res.plot_diagnostics(fig=fig, lags=30)
    fig = plt.figure(figsize=(16, 7))
    fig = res.plot_predict(0, 100, fig=fig)

## Autoregressive model

In [ ]:
model = AutoReg(df["value"], lags=1)
evaluate_model(model, df)

In [ ]:
model = AutoReg(df["value"], lags=4, seasonal=True)
evaluate_model(model, df)